# Creating and saving a single task dataset 

### Imports

In [1]:
import sys
sys.path.append('../')

In [2]:
from cognitive import constants as const
from cognitive.auto_task.auto_task_util import write_trial_instance
from cognitive.task_bank import CompareLocTemporal, CompareCategoryTemporal, SequentialCategoryMatch, SequentialLocationMatch
import os
import shutil
import random

2023-11-13 18:17:16.261170: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-11-13 18:17:16.261268: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-11-13 18:17:16.261345: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-11-13 18:17:16.267752: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-11-13 18:17:17.433630: W tensorflow/compiler/

### Constant Definitions

- Here we define the constants for the task

In [3]:
output_dir = '../datasets/val_big_2' # the output directory
stim_dir = '../data/new_shapenet_val' #../data/MULTIF_5_stim/MULTIF_5_stim' # stimulus set
n_tasks = 2000 # number of tasks to be generated
const.DATA = const.Data(dir_path=stim_dir)#, max_memory=3)

# Create the output directory
if os.path.exists(output_dir):
    shutil.rmtree(output_dir)
os.makedirs(output_dir)

Stimuli Directory:  ../data/new_shapenet_val
OrderedDict([((0, 0), [(0.0, 0.5), (0.0, 0.5)]), ((0, 1), [(0.0, 0.5), (0.5, 1.0)]), ((1, 0), [(0.5, 1.0), (0.0, 0.5)]), ((1, 1), [(0.5, 1.0), (0.5, 1.0)])])


- Generate the tasks and write them to the output directory
- (remember to specify 'train' parameter of write function)

In [4]:
for i in range(n_tasks):

    # Choose a task class (see task_bank or make your own see individual_task_gen.ipynb)
    task = CompareCategoryTemporal(whens=['last1','last0'])

    # Write trial to disk
    fp = os.path.join(output_dir, 'trial' + str(i))
    write_trial_instance(task, fp, 224, True, train=False)